In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

/home/isai/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
EPOCHS = 1000
STEPS_SIZE = 10

GRAPH_STEPS = EPOCHS / STEPS_SIZE

# x and y coordinates to plot 
x_coordinates = np.linspace(0, GRAPH_STEPS, num=GRAPH_STEPS)
loss_y_coordinates = np.zeros(GRAPH_STEPS)
accuracy_y_coordinates = np.zeros(GRAPH_STEPS)

def plot(x, y, limits, title, x_label_name, y_label_name):
    plt.close("all")
    figure = plt.figure()
    figure.clf()
    
    plt.plot(x, y)
    
    #plt.axis(limits)
    plt.ylabel(y_label_name)
    plt.xlabel(x_label_name)

    figure.suptitle(title, fontsize=20)
    plt.legend(loc='upper right')
    
    plt.show()
    return

def plot_loss():
    limits = [0,GRAPH_STEPS,0,5]
    plot(x_coordinates, loss_y_coordinates, limits, "Loss Overtime", "Epochs", "Loss")  
    return

def plot_accuracy():
    limits = [0,GRAPH_STEPS,0,1]
    plot(x_coordinates, accuracy_y_coordinates, limits, "Accuracy Overtime", "Epochs", "Accuracy")
    return

In [4]:
#sess = tf.InteractiveSession()

In [5]:
def weight_variable(shape):
    W_matrix = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(W_matrix)

def bias_variable(shape):
    b_vector = tf.constant(0.1, shape=shape)
    return tf.Variable(b_vector)

def conv2d(x, W_matrix):
    return tf.nn.conv2d(x, W_matrix, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [6]:
x = tf.placeholder(tf.float32, shape=(None, 784)
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())

for epoch in range(EPOCHS):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    if epoch % STEPS_SIZE == 0:
        #features_test, targets_test = mnist.test.images, mnist.test.labels
        #l_scalar = sess.run(cross_entropy, feed_dict={x:batch[0], y_: batch[1]})
        #a_scalar = sess.run(accuracy, feed_dict={x:batch[0], y_: batch[1]})
        l_scalar = sess.run(cross_entropy, feed_dict={x:batch[0], y_: batch[1]})
        a_scalar = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        
        
        loss_y_coordinates[epoch] = l_scalar
        accuracy_y_coordinates[epoch] = a_scalar
    

    print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

NameError: name 'sess' is not defined